## STEP 1. 형태소 분석기를 이용하여 품사가 명사인 경우 해당 단어를 추출하기

In [4]:
# 시간이 오래 소요되므로 파일로 저장
import os
from konlpy.tag import Okt

okt = Okt()
input_file_path = os.getenv('HOME') + '/aiffel/weat/synopsis.txt'
output_file_path = './tokenized/nouns.txt'

with open(input_file_path, 'r', encoding='utf-8') as rf:
    with open(output_file_path, 'w', encoding='utf-8') as wf:
        while True:
            line = rf.readline()
            if not line:
                break
            words = okt.pos(line, stem=True, norm=True)
            res = [w[0] for w in words if w[1] == "Noun"]
            wf.write(' '.join(res) + '\n')

FileNotFoundError: [Errno 2] No such file or directory: './tokenized/nouns.txt'

In [ ]:
with open(output_file_path, 'r', encoding='utf-8') as rf:
    for line in rf:
        # 각 줄을 읽어 공백을 기준으로 단어를 나누고 리스트로 변환
        words = line.strip().split()
        tokenized.append(words)

## STEP 2. 추출된 결과로 embedding model 만들기

In [3]:
from gensim.models import Word2Vec

# tokenized에 담긴 데이터를 가지고 나만의 Word2Vec을 생성합니다. (Gensim 4.0 기준)
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)  
model.wv.most_similar(positive=['영화'])

[('작품', 0.886648416519165),
 ('다큐멘터리', 0.8544517755508423),
 ('영화로', 0.824195146560669),
 ('드라마', 0.8082562685012817),
 ('코미디', 0.8076089024543762),
 ('시대극', 0.7985746264457703),
 ('주제', 0.7908185124397278),
 ('소재', 0.7800697684288025),
 ('감동', 0.7800178527832031),
 ('형식', 0.7779371738433838)]

## STEP 3. target, attribute 단어 셋 만들기

In [4]:
# TF/IDF로 해당 데이터를 가장 잘 표현하는 단어 셋 만들기
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from konlpy.tag import Okt

art_txt = 'synopsis_art.txt'
gen_txt = 'synopsis_gen.txt'

def read_token(file_name):
    okt = Okt()
    result = []
    with open(os.getenv('HOME')+'/aiffel/weat/'+file_name, 'r') as fread: 
        print(file_name, '파일을 읽고 있습니다.')
        while True:
            line = fread.readline() 
            if not line: break 
            tokenlist = okt.pos(line, stem=True, norm=True) 
            for word in tokenlist:
                if word[1] in ["Noun"]:#, "Adjective", "Verb"]:
                    result.append((word[0])) 
    return ' '.join(result)

synopsis_art.txt 파일을 읽고 있습니다.
synopsis_gen.txt 파일을 읽고 있습니다.


### target 단어 추출

In [ ]:
#파일 불러와서 TF-IDF 생성
art = read_token(art_txt)
gen = read_token(gen_txt)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([art, gen])

In [5]:
m1 = X[0].tocoo()   # art를 TF-IDF로 표현한 sparse matrix를 가져옵니다. 
m2 = X[1].tocoo()   # gen을 TF-IDF로 표현한 sparse matrix를 가져옵니다. 

# target 단어 추출
w1 = [[i, j] for i, j in zip(m1.col, m1.data)]
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 
w2.sort(key=lambda x: x[1], reverse=True)   #gen을 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 

print('예술영화를 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')
    
print('일반영화를 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

예술영화를 대표하는 단어들:
그녀, 자신, 시작, 위해, 사랑, 사람, 영화, 

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


친구, 남자, 가족, 이야기, 마을, 사건, 마음, 세상, 아버지, 아이, 엄마, 모든, 여자, 대한, 서로, 과연, 다시, 시간, 아들, 소녀, 아내, 다른, 사이, 영화제, 세계, 사실, 하나, 점점, 남편, 감독, 여행, 인생, 발견, 모두, 순간, 우리, 가장, 마지막, 생활, 아빠, 모습, 통해, 죽음, 기억, 비밀, 학교, 음악, 한편, 소년, 생각, 도시, 명의, 사고, 결혼, 전쟁, 때문, 위기, 이제, 최고, 이자, 과거, 일상, 경찰, 상황, 간다, 미국, 결심, 운명, 현실, 관계, 지금, 단편, 여인, 하루, 이름, 이후, 준비, 인간, 감정, 만난, 국제, 처음, 충격, 살인, 누구, 동안, 존재, 그린, 어머니, 연인, 계속, 동생, 작품, 

일반영화를 대표하는 단어들:
자신, 그녀, 영화제, 위해, 사람, 시작, 국제, 영화, 친구, 사랑, 남자, 이야기, 대한, 서울, 여자, 사건, 남편, 아이, 가족, 아버지, 다른, 마을, 시간, 엄마, 아들, 모든, 단편, 마음, 사실, 다시, 세계, 모습, 작품, 통해, 생각, 서로, 세상, 발견, 감독, 아내, 관계, 소녀, 사이, 하나, 우리, 애니메이션, 때문, 여성, 죽음, 과연, 점점, 인간, 생활, 한편, 결혼, 상황, 모두, 기억, 명의, 소년, 여행, 가장, 간다, 순간, 이제, 도시, 비밀, 학교, 과거, 가지, 이자, 경찰, 마지막, 미국, 동안, 전쟁, 주인공, 대해, 존재, 현실, 연출, 사고, 살인, 일상, 어머니, 계속, 사회, 인생, 다큐멘터리, 부문, 섹스, 최고, 바로, 동생, 의도, 하루, 위기, 계획, 정체, 한국, 

In [ ]:
# 중복 제거
n = 15
w1_, w2_ = [], []
for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break 

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break

### attribute 단어 추출 

In [ ]:
#파일 불러와서 TF-IDF 생성
genre_txt = ['synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt', 
             'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt', 
             'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt', 
             'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt', 
             'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt']
genre_name = ['SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)',
         '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지']
genre = []
for file_name in genre_txt:
    genre.append(read_token(file_name))
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(genre)

In [6]:
m = [X[i].tocoo() for i in range(X.shape[0])]

w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)
attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) < 15):
        if vectorizer.get_feature_names()[w[i][j][0]] in model.wv:
            attr.append(vectorizer.get_feature_names()[w[i][j][0]])
            print(vectorizer.get_feature_names()[w[i][j][0]], end=', ')
        j += 1
    attributes.append(attr)
    print()

synopsis_SF.txt 파일을 읽고 있습니다.
synopsis_family.txt 파일을 읽고 있습니다.
synopsis_show.txt 파일을 읽고 있습니다.
synopsis_horror.txt 파일을 읽고 있습니다.
synopsis_etc.txt 파일을 읽고 있습니다.
synopsis_documentary.txt 파일을 읽고 있습니다.
synopsis_drama.txt 파일을 읽고 있습니다.
synopsis_romance.txt 파일을 읽고 있습니다.
synopsis_musical.txt 파일을 읽고 있습니다.
synopsis_mystery.txt 파일을 읽고 있습니다.
synopsis_crime.txt 파일을 읽고 있습니다.
synopsis_historical.txt 파일을 읽고 있습니다.
synopsis_western.txt 파일을 읽고 있습니다.
synopsis_adult.txt 파일을 읽고 있습니다.
synopsis_thriller.txt 파일을 읽고 있습니다.
synopsis_animation.txt 파일을 읽고 있습니다.
synopsis_action.txt 파일을 읽고 있습니다.
synopsis_adventure.txt 파일을 읽고 있습니다.
synopsis_war.txt 파일을 읽고 있습니다.
synopsis_comedy.txt 파일을 읽고 있습니다.
synopsis_fantasy.txt 파일을 읽고 있습니다.
SF: 위해, 자신, 지구, 시작, 사람, 인류, 인간, 미래, 우주, 그녀, 로봇, 세계, 모든, 박사, 우주선, 
가족: 엄마, 아빠, 가족, 영화제, 자신, 위해, 친구, 아주르, 아버지, 시작, 그녀, 아들, 마을, 국제, 낙타, 
공연: 오페라, 사랑, 토스카, 실황, 올레, 자신, 카바, 그녀, 공연, 오텔로, 리골레토, 백작, 프레, 베르디, 위해, 
공포(호러): 시작, 위해, 사람, 자신, 친구, 그녀, 사건, 공포, 발견, 죽음, 마을, 가족, 악령, 남자, 좀비, 
기타: 영화제, 국제, 서울, 단편, 영화, 자

## STEP 4. WEAT score 계산과 시각화

In [10]:
from numpy import dot
from numpy.linalg import norm
# WEAT score function
def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))
def s(w, A, B):
    c_a = cos_sim(w, A)
    c_b = cos_sim(w, B)
    mean_A = np.mean(c_a, axis=-1)
    mean_B = np.mean(c_b, axis=-1)
    return mean_A - mean_B #, c_a, c_b
def weat_score(X, Y, A, B):
    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

In [11]:
# embedding model과 단어 셋으로 WEAT score 구해보기
matrix = [[0 for _ in range(len(genre_name))] for _ in range(len(genre_name))]
X = np.array([model.wv[word] for word in target_art])
Y = np.array([model.wv[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model.wv[word] for word in attributes[i]])
        B = np.array([model.wv[word] for word in attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)
        
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print(genre_name[i], genre_name[j],matrix[i][j])

SF 가족 -0.5250815
SF 공연 -0.38076147
SF 공포(호러) -0.6964674
SF 기타 0.35148215
SF 다큐멘터리 0.5931078
SF 드라마 -0.38868818
SF 멜로로맨스 -0.8079767
SF 뮤지컬 0.32724223
SF 미스터리 -0.7034123
SF 범죄 -0.26127744
SF 사극 -0.836256
SF 서부극(웨스턴) -0.42475638
SF 성인물(에로) -0.58250743
SF 스릴러 -0.5395713
SF 애니메이션 0.36044207
SF 액션 -0.4819872
SF 어드벤처 -0.8033882
SF 전쟁 0.22449192
SF 코미디 -0.45639727
SF 판타지 -0.20104498
가족 공연 0.20113707
가족 공포(호러) -0.17772157
가족 기타 0.75772876
가족 다큐멘터리 0.8397989
가족 드라마 0.22676712
가족 멜로로맨스 -0.63209766
가족 뮤지컬 0.7046117
가족 미스터리 -0.32887682
가족 범죄 0.18669163
가족 사극 -0.08369598
가족 서부극(웨스턴) 0.445031
가족 성인물(에로) -0.32356948
가족 스릴러 -0.15422046
가족 애니메이션 0.8597249
가족 액션 0.08878305
가족 어드벤처 -0.022493953
가족 전쟁 0.66804725
가족 코미디 0.1990872
가족 판타지 0.4279034
공연 공포(호러) -0.26817703
공연 기타 0.934007
공연 다큐멘터리 0.9179254
공연 드라마 0.0092852535
공연 멜로로맨스 -0.73806673
공연 뮤지컬 0.87697643
공연 미스터리 -0.3829619
공연 범죄 0.05396664
공연 사극 -0.39449522
공연 서부극(웨스턴) 0.049652636
공연 성인물(에로) -0.5053268
공연 스릴러 -0.239368
공연 애니메이션 0.9082278
공연 액션 -0.05486

In [2]:
# 시각화
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def plot_heatmap(matrix, genre_name, title, save=True):
    np.random.seed(0)
    # 한글 지원 폰트
    sns.set(font='NanumGothic')

    # 마이너스 부호 
    plt.rcParams['axes.unicode_minus'] = False

    fig, ax = plt.subplots(figsize=(15,13))
    sns.heatmap(matrix, xticklabels=genre_name, yticklabels=genre_name, annot=True, cmap='RdYlGn_r', ax=ax)

    ax.set_title(title, fontsize=20, pad=20)
    if save:
        plt.savefig('heatmap_'+title+'.png', bbox_inches='tight')

    plt.show()
    
plot_heatmap(matrix, genre_name, 'simple_tfidf', False)

NameError: name 'matrix' is not defined